In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pickle
import random
import requests
import io
import zipfile
from scipy.stats import norm

import cvxpy as cp


In [2]:

def compute_monthly_factor_returns_from_daily(daily_factors: pd.DataFrame, factor_cols=None):
    """
    Input: simple daily returns (not log). Monthly return is multiplicative.
    Output index = last *trading* day available in the data for each month
    (so it matches weights/costs indexed by last trading day).
    # TODO: in this simulation we have log so we need to change
    """
    if factor_cols is None:
        factor_cols = list(daily_factors.columns)

    x = daily_factors[factor_cols].copy().dropna(how="all")
    month = x.index.to_period("M")

    monthly = (1.0 + x).groupby(month).prod() - 1.0

    # index = last trading day in each month 
    assert x.index.is_monotonic_increasing
    month_end_dates = x.groupby(month).tail(1).index
    monthly.index = month_end_dates

    return monthly.sort_index()
    


In [3]:
# we need daily and not monthly returns
def daily_portfolio_returns_from_monthly_weights(daily_returns: pd.DataFrame,
                                                 W_monthly: pd.DataFrame) -> pd.Series:

    R = daily_returns.dropna(how="any").copy()
    month = R.index.to_period("M")

    W = W_monthly.copy()
    W["m"] = W.index.to_period("M")
    W = W.set_index("m").sort_index()

    out = []
    months = sorted(month.unique())
    for m_trade in months:
        if m_trade not in W.index:
            continue

        w = W.loc[m_trade]
        cols = [c for c in w.index if c in R.columns]
        Rt = R.loc[month == m_trade, cols]
        if Rt.empty:
            continue

        out.append(pd.Series(Rt.values @ w[cols].values, index=Rt.index))

    return pd.concat(out).sort_index()


def portfolio_stats_paper_style(returns,
                                periods_per_year=252,
                                rf_annual=0.0,
                                target=0.0,
                                alpha=0.95):

    r = pd.Series(returns).dropna().astype(float).to_numpy()
    if len(r) < 2:
        raise ValueError("need at least 2 observations")
    if not (0.0 < alpha < 1.0):
        raise ValueError("alpha must be in (0,1)")

    mu = float(np.mean(r))
    sigma2 = float(np.var(r, ddof=0))
    sigma = float(np.sqrt(sigma2))

    mu_ann = mu * periods_per_year
    sigma2_ann = sigma2 * periods_per_year
    sigma_ann = float(np.sqrt(sigma2_ann))

    downside = (r[r < target] - target)
    semivar = 0.0 if downside.size == 0 else float(np.mean(downside**2))
    semidev_ann = float(np.sqrt(semivar * periods_per_year))

    # VaR/CVaR
    q = 1.0 - alpha
    VaR = float(np.quantile(r, q))
    tail = r[r <= VaR]
    CVaR = VaR if tail.size == 0 else float(np.mean(tail))

    # drawdowns
    wealth = np.concatenate([[1.0], np.cumprod(1.0 + r)])
    peak = np.maximum.accumulate(wealth)
    dd = 1.0 - wealth / peak
    pos_dd = dd[dd > 0]
    avg_dd = 0.0 if pos_dd.size == 0 else float(np.mean(pos_dd))

    # excess mean (per-period rf from annual rf)
    rf_per = (1.0 + rf_annual)**(1.0 / periods_per_year) - 1.0
    excess_ann_mean = (mu - rf_per) * periods_per_year

    sharpe = np.nan if sigma_ann == 0 else float(excess_ann_mean / sigma_ann)
    sortino = np.nan if semidev_ann == 0 else float(excess_ann_mean / semidev_ann)

    # tail-adjusted Sharpe (NO annualization of CVaR/mVaR)
    ta_sharpe_cvar = np.nan if CVaR == 0 else float(excess_ann_mean / abs(CVaR))

    # Cornish-Fisher modified VaR
    if sigma == 0:
        skew = 0.0
        exkurt = 0.0
    else:
        xc = r - mu
        m3 = float(np.mean(xc**3))
        m4 = float(np.mean(xc**4))
        skew = m3 / (sigma**3)
        kurt = m4 / (sigma**4)
        exkurt = kurt - 3.0

    z = float(norm.ppf(q))
    z_cf = (z
            + (1/6)  * (z**2 - 1)   * skew
            + (1/24) * (z**3 - 3*z) * exkurt
            - (1/36) * (2*z**3 - 5*z) * (skew**2))

    mVaR = float(mu + sigma * z_cf)
    ta_sharpe_mvar = np.nan if mVaR == 0 else float(excess_ann_mean / abs(mVaR))

    return {
        "Ann. Mean (%)": 100 * mu_ann,
        "Ann. StdDev (%)": 100 * sigma_ann,
        "Ann. SemiDev (%)": 100 * semidev_ann,
        "CVaR 95% (%)": 100 * CVaR,
        "Avg DD (%)": 100 * avg_dd,
        "VaR 95% (%)": 100 * VaR,
        "Sharpe (ann.)": sharpe,
        "Sortino (ann.)": sortino,
        "Tail-Adj Sharpe (CVaR95)": ta_sharpe_cvar,
        "Tail-Adj Sharpe (mVaR95)": ta_sharpe_mvar,
    }


def make_table_for_portfolios(portfolios: dict,
                              periods_per_year=252,
                              rf_annual=0.0,
                              target=0.0,
                              alpha=0.95) -> pd.DataFrame:
    rows = [
        "Ann. Mean (%)",
        "Ann. StdDev (%)",
        "Ann. SemiDev (%)",
        "CVaR 95% (%)",
        "Avg DD (%)",
        "VaR 95% (%)",
        "Sharpe (ann.)",
        "Sortino (ann.)",
        "Tail-Adj Sharpe (CVaR95)",
        "Tail-Adj Sharpe (mVaR95)",
    ]

    out = pd.DataFrame(index=rows)
    for name, r in portfolios.items():
        st = portfolio_stats_paper_style(r, periods_per_year=periods_per_year,
                                         rf_annual=rf_annual, target=target, alpha=alpha)
        out[name] = [st[k] for k in rows]
    return out

def df_to_booktabs_latex(df: pd.DataFrame, caption=None, label=None) -> str:
    latex = df.to_latex(
        escape=True,
        float_format=lambda x: f"{x:.2f}",
        column_format="l" + "r"*df.shape[1],
        bold_rows=False
    )
    # convert to booktabs style
    latex = latex.replace("\\toprule", "\\toprule").replace("\\midrule", "\\midrule").replace("\\bottomrule", "\\bottomrule")
    if caption or label:
        # wrap in table environment if requested
        body = latex
        lines = ["\\begin{table}[!htbp]", "\\centering"]
        if caption:
            lines.append(f"\\caption{{{caption}}}")
        if label:
            lines.append(f"\\label{{{label}}}")
        lines.append(body.strip())
        lines.append("\\end{table}")
        latex = "\n".join(lines)
    return latex

In [4]:
# with turnover cost
# do not do daily evaulation and cost. simple monthly!!
def Markowitz_with_turnover_TC_diffobj(
    daily_factors, factor_cols,
    gamma=5.0, ridge=1e-8, nonneg=False,
    market_vol_proxy="Mkt-rf",
    c_tc=0.0021, abs_eps=1e-10,
    use_drift_turnover=True,              # TRUE = drift turnover (RL,Vol paper style); FALSE = no costs at all
    bounded_b: bool = False
                        ):

    _printed = False

    def _check_month_loss_once(monthly_F, rv2_aligned):
        nonlocal _printed
        if _printed:
            return
        _printed = True

        months_all = monthly_F.index.to_period("M")
        months_valid = rv2_aligned[rv2_aligned.notna()].index.to_period("M")

        lost = pd.Index(months_all).difference(pd.Index(months_valid))


    factor_cols = [c for c in factor_cols if c != market_vol_proxy] # for optimization (input to markowitz), safety

    monthly_F = compute_monthly_factor_returns_from_daily(daily_factors, factor_cols=factor_cols) # monthly returns, do not include MKT-RF
    m = daily_factors.dropna().index.to_period("M")

    # realized volatility on Mkt-rf, we ll use it for scaling
    # If market_vol_proxy column doesn't exist (simulated data), create equal-weighted market index
    if market_vol_proxy not in daily_factors.columns:
        # Create equal-weighted market index from available assets
        # effect?!!!!!!!!!!!
        market_index = daily_factors[factor_cols].mean(axis=1)
        daily_factors_with_mkt = daily_factors.copy()
        daily_factors_with_mkt[market_vol_proxy] = market_index
        mkt_col = market_vol_proxy# ....
        rv = daily_factors_with_mkt[mkt_col].groupby(m).std(ddof=1)
    else:
        mkt_col = market_vol_proxy
        rv = daily_factors[mkt_col].groupby(m).std(ddof=1)
    
    rv2 = rv.shift(1)  # use lagged volatility (no look-ahead)

    #align lagged vol to monthly_F using MONTH PERIOD, then restore trading month-end dates 
    rv2 = rv2.reindex(monthly_F.index.to_period("M")) # it
    rv2.index = monthly_F.index
    assert rv2.index.equals(monthly_F.index)


    _check_month_loss_once(monthly_F, rv2)


    # align and drop months where lagged vol is missing
    valid = rv2.notna()
    Rm = monthly_F.loc[valid]
    s = rv2.loc[valid].to_numpy().reshape(-1, 1)# we need to reshape cause of the division part

    R = Rm.to_numpy()  # (T x K)
    K = R.shape[1]
    T = R.shape[0]
    s = np.maximum(s, 1e-12)  # safety
    ### we use these above R,s inside the objective function and
    def smooth_abs(x):
        return np.sqrt(x*x + abs_eps)

    def eta_to_theta(eta):
        a = eta[:K]
        b = eta[K:]
        theta = a[None, :] + b[None, :] / s # already lagged
        # Normalize to sum to 1
        denom = np.sum(theta, axis=1, keepdims=True)
        theta = theta / np.where(np.abs(denom) > 1e-12, denom, 1.0)
        return theta

    def taus_series(eta):
        # TRUE = drift turnover (RL-style pre-trade drift turnover)
        # FALSE = do not even calculate costs: if we do not calculate costs, we should stuck to long-short also
        if not use_drift_turnover:
            return np.zeros(R.shape[0], dtype=float)

        theta = eta_to_theta(eta)
        T = theta.shape[0]
        taus = np.zeros(T, dtype=float)
        if T <= 1:
            return taus

        for t in range(1, T):
            # last month target (post-trade)
            w_prev_post = theta[t-1]

            # realized monthly returns during last month
            R_prev = R[t-1]

            # drift to pre-trade weights
            g = 1.0 + R_prev
            numer = w_prev_post * g
            denom = float(np.sum(numer))
            w_pre = numer / denom if abs(denom) > 1e-12 else w_prev_post

            # trade to current target
            w_target = theta[t]

            # turnover
            tau_t = 0.5 * float(np.sum(smooth_abs(w_target - w_pre)))
            taus[t] = tau_t # save monthly costs--> use it for net retunrs, we do not multiply with c_tc here!
            # later for cost calculation you need

        return taus

    def TC(eta):
        # TRUE = drift turnover (RL-style pre-trade drift turnover)
        # FALSE = do not even calculate costs: if we do not calculate costs, we should stuck to long-short also
        if not use_drift_turnover:
            return 0.0

        taus = taus_series(eta)
        return c_tc * float(np.mean(taus[1:])) if taus.shape[0] > 1 else 0.0 # no drift in the beginning

    def objective(eta):
        theta = eta_to_theta(eta)
        rp = np.sum(theta * R, axis=1)      # monthly portfolio return series, no R_ext!!!
        mu_p = float(rp.mean())
        var_p = float(rp.var(ddof=0))
        return 0.5 * gamma * var_p - mu_p + TC(eta)

    # initial guess
    guess = np.ones(2 * K) / (2 * K)

    constraints = []  # keep empty (no equality constraint): add up to one effect?

    # If unconditional benchmark (no timing): force b = 0 AND enforce sum(a)=1 (fully invested)
    # Fully invested constraint is needed for valid cost calculation (drift assumes normalized weights)
    if bounded_b:
        constraints.append({"type": "eq", "fun": lambda eta: np.sum(eta[:K]) - 1.0})  # sum(a) = 1

    # bounds: if nonneg True => both a,b >= 0; if False, a >= 0 but b unbounded for volatility timing
    if nonneg:
        bounds = [(0.0, None)] * (2 * K)  # Both a and b >= 0
    else:
        bounds = [(0.0, None)] * K + [(None, None)] * K  # a >= 0, b unbounded

    # If unconditional benchmark (bounded_b=True): force b = 0
    if bounded_b:
        bounds = [(0, None)] * K + [(0, 0)] * K  # Force all b values to exactly 0
    result = minimize(
        objective,
        x0=guess,
        bounds=bounds,
        constraints=constraints,
        method='SLSQP',
        options={'maxiter': 100000}
    )

    eta = result.x
    a = eta[:K]
    b = eta[K:]

    theta = eta_to_theta(eta)
    weights = pd.DataFrame(theta, index=Rm.index, columns=Rm.columns) # Rm.index cause

    # portfolio return in month t: sum_k w_{k,t} * R_{k,t}
    port_ret_gross = (weights * Rm).sum(axis=1)

    # Substract the costs (month-by-month, consistent with drift turnover)
    taus = taus_series(eta) # just
    costs = c_tc * taus # monthly
    port_ret_net = port_ret_gross - pd.Series(costs, index=Rm.index)

    return {
        "a": pd.Series(a, index=Rm.columns, name="a"),
        "b": pd.Series(b, index=Rm.columns, name="b"),
        "weights": weights,
        "portfolio_returns": port_ret_gross,
        "portfolio_returns_gross": port_ret_gross,
        "portfolio_returns_net": port_ret_net,
        "turnover": pd.Series(taus, index=Rm.index, name="turnover"),
        "costs": pd.Series(costs, index=Rm.index, name="costs"), # monthly costs
        "opt_result": result,
        "TC_in_sample": TC(eta),
        "use_drift_turnover": use_drift_turnover,
        "bounded_b": bounded_b
    }

In [5]:

def set_numpy_determinism(seed=0):
    '''
    Similarly to RL set up random sets and go over a list of seeds
    '''
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)




def simulate_markov_chain(Q, T, k0=0, rng=None):
    if rng is None:
        rng = np.random.default_rng(0)
    Q = np.asarray(Q, float)
    K = Q.shape[0]
    k = np.empty(T, dtype=int)
    k[0] = int(k0)
    for t in range(1, T):
        k[t] = rng.choice(K, p=Q[k[t - 1]]) # 0,3,1 etc . which regime:)
    return k


def multivariate_t_eps_with_target_cov(rng, Sigma, df: float):
    """
    eps ~ multivariate t with df, scaled so Cov(eps) = Sigma (for df>2).
    Construction: z~N(0,Sigma), scale by s = sqrt(df/chi2_df),
    then multiply by sqrt((df-2)/df) to keep covariance = Sigma.
    """
    Sigma = np.asarray(Sigma, float)
    z = rng.multivariate_normal(mean=np.zeros(Sigma.shape[0]), cov=Sigma)
    g = rng.chisquare(df)
    s = np.sqrt(df / g)
    eps = z * s * np.sqrt((df - 2.0) / df)  # ensures Cov(eps)=Sigma
    return eps


def simulate_rs_var1_monthly_regimes_RS_SV_T(
    T_days,
    Q,
    c_list,
    Phi,
    Sigma_list,
    k0=0,
    burn_in_months=50,
    rng=None,
    start_date="2000-01-03",
    df_list=None,                 # Student-t df per regime
    sv_rho=0.97,                  # persistence of log-vol (vol clustering)
    sv_sigma=0.20,                # innovation std of log-vol
    logh_mu_list=None,            # regime-specific mean level of log-vol
):
    """
    Regime-switching VAR(1) with:
      - monthly Markov regimes
      - daily VAR(1)
      - Student-t shocks (fat tails)
      - persistent stochastic volatility scale h_t (vol clustering)
    https://en.wikipedia.org/wiki/Multivariate_t-distribution
    
    Conditional shock covariance: h_t * Sigma_list[regime]
    """
    if rng is None:
        rng = np.random.default_rng(0)

    Q = np.asarray(Q, float)
    c_list = np.asarray(c_list, float)
    Sigma_list = np.asarray(Sigma_list, float)

    K = Q.shape[0]
    N = c_list.shape[1]

    Phi_arr = np.asarray(Phi, float)
    if Phi_arr.ndim == 2:
        Phi_arr = np.repeat(Phi_arr[None, :, :], K, axis=0) # rethink this! we may change this in the example !!!!!!

    # defaults
    if df_list is None:
        df_list = np.full(K, 8.0)  # moderately fat tails everywhere
    df_list = np.asarray(df_list, float)

    if logh_mu_list is None:
        # baseline: low vol in bull, medium in neutral, high in bear #  we may change bull neautr\l
        # interpreted as log(h), where h multiplies Sigma
        logh_mu_list = np.array([-2.5, -1.5, -0.5], dtype=float)[:K]
    logh_mu_list = np.asarray(logh_mu_list, float)

    # Build calendar sample dates
    sample_dates = pd.bdate_range(start_date, periods=T_days)
    start_month = sample_dates[0].to_period("M")
    end_month   = sample_dates[-1].to_period("M")

    sample_months = pd.period_range(start=start_month, end=end_month, freq="M")
    T_months = len(sample_months)

    full_months = pd.period_range(start=(start_month - burn_in_months), end=end_month, freq="M") # we wont use burn in months
    TT_months = len(full_months)

    # simulate monthly regimes
    k_month_full = simulate_markov_chain(Q, TT_months, k0=k0, rng=rng)

    # expand to daily regimes aligned to business days
    all_dates_full = []
    k_days_full = []

    for m_idx, per in enumerate(full_months):
        month_start = per.to_timestamp(how="start")
        month_end   = per.to_timestamp(how="end")
        dts = pd.bdate_range(month_start, month_end)

        all_dates_full.append(dts)
        k_days_full.append(np.full(len(dts), int(k_month_full[m_idx]), dtype=int)) # every business day in a month inherits that month’s regime

    all_dates_full = all_dates_full[0].append(all_dates_full[1:]) if len(all_dates_full) > 1 else all_dates_full[0]
    k_days_full = np.concatenate(k_days_full, axis=0)

    # simulate daily log returns with SV scale + t innovations
    r_full = np.zeros((len(all_dates_full), N), dtype=float)

    # initialize log-vol
    k0_day = int(k_days_full[0])
    logh = float(logh_mu_list[k0_day])  # start at regime mean

    for t in range(1, len(all_dates_full)):
        kt = int(k_days_full[t])
        df = float(df_list[kt])

        # stochastic volatility (one-factor scale)
        # log h_t = mu_k + rho*(log h_{t-1} - mu_k) + sigma * eta_t
        logh = (logh_mu_list[kt] +
                sv_rho * (logh - logh_mu_list[kt]) +
                sv_sigma * rng.standard_normal())
        h = float(np.exp(logh))  # scale multiplier on Sigma

        # heavy-tailed shock with target covariance Sigma_list[kt]
        eps = multivariate_t_eps_with_target_cov(rng, Sigma_list[kt], df=df)

        # apply SV scale (now Cov = h * Sigma_k)
        eps = np.sqrt(h) * eps

        r_full[t] = c_list[kt] + Phi_arr[kt] @ r_full[t - 1] + eps

    # drop burn-in by date indexing
    pos = all_dates_full.get_indexer(sample_dates)
    r_days = r_full[pos]
    k_days = k_days_full[pos]
    k_month = k_month_full[-T_months:]
    # helper: make sure that dates logic alogn

    return r_days, k_days, k_month, sample_dates, sample_months

def mv_simple(
    daily_ret: pd.DataFrame,
    cols=None,
    gamma = 5.0,
    ridge: float = 1e-10,
    sum_to_one_constraint: bool = True,  # True => enforce sum(w)=1; False => no equality constraint
    long_only: bool = True,              # True => bounds [0,1]; False => no bounds
    maxiter: int = 10000,
                        ):
    if cols is not None:
        daily_ret = daily_ret[cols]
    # monthly
    Rm_df = compute_monthly_factor_returns_from_daily(daily_ret) # monthly
    cols = list(Rm_df.columns)

    R = Rm_df.to_numpy()
    T, K = R.shape

    Sigma = np.cov(R, rowvar=False) + ridge * np.eye(K)
    Return = np.mean(R, axis=0)

    def objective(w):
        w = np.asarray(w, float)
        return float(0.5 * gamma * (w @ Sigma @ w) - (w @ Return))

    constraints = []
    if sum_to_one_constraint:
        constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0})

    if long_only:
        bounds = [(0.0, 1)] * K
    else:
        bounds = None  # not bounding is false => no bounds at all

    x0 = np.ones(K) / K  # simple start

    res = minimize(
        objective,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": maxiter},
    )

    w = np.asarray(res.x, float)
    w = pd.Series(w, index=cols, name="w_mv")

    # monthly portfolio return series (gross, no cost)
    port_monthly = pd.Series(Rm_df.to_numpy() @ w.to_numpy(), index=Rm_df.index, name="port_mv")

    return {
        "weights": w,
        "monthly_returns": Rm_df,
        "port_monthly": port_monthly,
        "Sigma": Sigma,
        "opt_result": res,
        "sum_to_one_constraint": sum_to_one_constraint,
        "long_only": long_only,
    }



def mw_cvar_simple(
    daily_ret: pd.DataFrame,
    cols=None,
    beta: float = 0.95,                 # CVaR confidence level
    gamma: float = 1.0,                 # tradeoff weight on CVaR: 1!
    sum_to_one_constraint: bool = True, # enforce sum(w)=1
    long_only: bool = True,             # enforce w>=0
    upper_bound: float | None = 1.0,    # set None to remove w<=upper_bound
    solver: str = "ECOS",
    verbose: bool = False,
        ):
    if cols is not None:
        daily_ret = daily_ret[cols]

    # monthly returns
    Rm_df = compute_monthly_factor_returns_from_daily(daily_ret)
    asset_cols = list(Rm_df.columns)

    R = Rm_df.to_numpy()  # (T, K)
    T, K = R.shape
    print(f"R shape: T (months) = {T:,},  K (assets) = {K:,}")

    mu_vec = R.mean(axis=0)  # (K,)

    # CVXPY variables
    w = cp.Variable(K)
    alpha = cp.Variable()      # VaR-like threshold
    u = cp.Variable(T)         # tail slacks

    # losses = - portfolio return
    losses = -(R @ w)          # (T,)

    # CVaR(loss) definition
    cvar = alpha + (1 / ((1 - beta) * T)) * cp.sum(u)

    constraints = [
        u >= 0,
        u >= losses - alpha
    ]

    if sum_to_one_constraint:
        constraints.append(cp.sum(w) == 1)

    if long_only:
        constraints.append(w >= 0)

    if upper_bound is not None:
        constraints.append(w <= upper_bound)

    # minimize: gamma * CVaR(loss) - expected_return
    objective = cp.Minimize(gamma * cvar - mu_vec @ w)

    prob = cp.Problem(objective, constraints)
    prob.solve(solver=solver, verbose=verbose)

    if prob.status not in ("optimal", "optimal_inaccurate"):
        raise RuntimeError(f"CVaR optimization failed. Status: {prob.status}")

    w_val = np.asarray(w.value, float).reshape(-1)
    w_ser = pd.Series(w_val, index=asset_cols, name="w_mw_cvar")

    # monthly portfolio return series
    port_monthly = pd.Series(Rm_df.to_numpy() @ w_ser.to_numpy(),
                             index=Rm_df.index,
                             name="port_mw_cvar")

    out = {
        "weights": w_ser,
        "monthly_returns": Rm_df,
        "port_monthly": port_monthly,
        "mu_vec": pd.Series(mu_vec, index=asset_cols, name="mu"),
        "beta": beta,
        "gamma": gamma,
        "cvar_loss": float(cvar.value),
        "var_alpha": float(alpha.value),
        "expected_return": float(mu_vec @ w_val),
        "opt_status": prob.status,
        "opt_value": float(prob.value),
        "problem": prob,   # keep if you want to inspect duals etc.
        "sum_to_one_constraint": sum_to_one_constraint,
        "long_only": long_only,
        "upper_bound": upper_bound,
    }
    return out


In [6]:
seed_list = [89, 53, 274, 1234]  
regime_names = ["Bull", "Neutral", "Bear"]
df_list = np.array([20.0, 10.0, 5.0])
logh_mu_list = np.array([-1.6, -1.3, -0.7])
sv_rho, sv_sigma = 0.97, 0.20


# transition matrix matter more i think if we have enough data.
Phi_fixed = np.array([[0.15, 0.10],
                      [0.10, 0.15]])

c_list = np.array([
    [0.0040, 0.0030],   # Bull
    [0.0030, 0.0028],   # Neutral
    [-0.0090, 0.0030],  # Bear
])

Sigma_list = np.array([
    [[0.0005, 0.00010],
     [0.00010, 0.00045]],   # Bull
    [[0.0018, 0.0],
     [0.0,    0.0014]],     # Neutral
    [[0.0050, -0.0030],
     [-0.0030, 0.0020]],    # Bear
])

Q_bull_bear = np.array([
    [0.74, 0.02, 0.24],
    [0.10, 0.82, 0.08],
    [0.30, 0.02, 0.68],
])

Q_neutral_bear = np.array([
    [0.82, 0.08, 0.10],
    [0.02, 0.68, 0.30],
    [0.02, 0.24, 0.74],
])

Q_bull_neutral = np.array([
    [0.74, 0.24, 0.02],
    [0.30, 0.68, 0.02],
    [0.10, 0.08, 0.82],
])
scenarios = [
    ("Bull-Bear", Q_bull_bear),
    ("Neutral-Bear", Q_neutral_bear),
    ("Bull-Neutral", Q_bull_neutral),
]

results = {}


In [7]:
def check_phi_stability(Phi, tol=1.0 - 1e-8):
    Phi = np.asarray(Phi, float)
    if Phi.ndim == 2:
        eigs = np.linalg.eigvals(Phi)
        max_abs = float(np.max(np.abs(eigs)))
        if max_abs >= tol:
            raise RuntimeError(f"Unstable VAR: max |eig| = {max_abs:.6f} >= 1.0")
        print(f"Phi stable: max |eig| = {max_abs:.6f}")
        return
    if Phi.ndim == 3:
        for k in range(Phi.shape[0]):
            eigs = np.linalg.eigvals(Phi[k])
            max_abs = float(np.max(np.abs(eigs)))
            if max_abs >= tol:
                raise RuntimeError(f"Unstable VAR in regime {k}: max |eig| = {max_abs:.6f} >= 1.0")
        print("Phi stable in all regimes.")
        return
    raise ValueError("Phi must be 2D or 3D")

# Check current Phi_fixed
check_phi_stability(Phi_fixed)

Phi stable: max |eig| = 0.250000


In [8]:
# make a table


In [9]:
# opening original files
tau_levels = [0.1,0.5,0.9] 
tau_levels_str = [str(tau).replace('.', '') for tau in tau_levels]
seeds = [53,274,1234,89] # seem seeds before

#base_path ="C:/Users/95att/Desktop/job/First_paper_QAC/Model based RL Gauss/training_outcome/"
base_path = "C:/Users/95att/Desktop/job/First_paper_QAC/Model based RL Dirichlet/training_outcome/"

In [10]:

def load_seed_outputs(
    base_path,
    tau_levels=(0.1, 0.5, 0.9),
    seed_list=(53,274,1234,89),
    folder_template="20250911_final_weighted_q_spwise_standard_tanh_{seed}_{tau_str}",
    filename="snapshot_train_end_summary_full.pkl",
                        ):

    base_path = Path(base_path) # Creating a path object
    tau_levels_str = [str(t).replace(".", "") for t in tau_levels]

    out, paths = {}, {}
    for tau_str in tau_levels_str:
        out[tau_str] = {}
        paths[tau_str] = {}
        for seed in seed_list:
            folder_name = folder_template.format(seed=seed, tau_str=tau_str)
            fpath = base_path / folder_name / filename
            if not fpath.exists():
                print(f"File not found: {fpath}")
                continue
            with open(fpath, "rb") as fh:
                payload = pickle.load(fh)
            out[tau_str][seed] = payload
            paths[tau_str][seed] = str(fpath)
    return out, paths#

In [11]:
nested_bull_bear, found_paths_bull_bear = load_seed_outputs(
    base_path=base_path,
    tau_levels=tau_levels,
    seed_list=seeds,
    folder_template="final_Q_bull_bear_original_{seed}_{tau_str}",
    filename="snapshot_train_end_summary_full.pkl",
)
nested_bull_neutral, found_paths_bull_bear = load_seed_outputs(
    base_path=base_path,
    tau_levels=tau_levels,
    seed_list=seeds,
    folder_template="final_Q_bull_neutral_original_{seed}_{tau_str}",
    filename="snapshot_train_end_summary_full.pkl",
)

nested_neutral_bear, found_paths_bull_bear = load_seed_outputs(
    base_path=base_path,
    tau_levels=tau_levels,
    seed_list=seeds,
    folder_template="final_Q_neutral_bear_original_{seed}_{tau_str}",
    filename="snapshot_train_end_summary_full.pkl",
)


In [12]:
# TODO:

# In-Sample Simulation Framework (No Saving)

This section simulates Markov regime-switching returns and evaluates strategies.
Outputs are displayed only (no files are saved).

In [ ]:

class MarkovSimulationEnvironment:
    # TODO: loop over seed and test seeds, align with RL
    def __init__(self, Q, c_list, Phi, Sigma_list, regime_names,
                 T_days=252*50, seed=123, burn_in_months=50,
                 start_date="2000-01-03", k0=0, asset_names=None,
                 df_list=None, sv_rho=0.97, sv_sigma=0.20, logh_mu_list=None,
                 verbose: bool = False):
        """
        Q: Transition matrix (K x K)
        c_list: Drift per regime (K x N)
        Phi: AR(1) coefficient matrix (N x N) or (K x N x N)
        Sigma_list: Covariance per regime (K x N x N)
        regime_names: List of regime names
        df_list: Student-t df per regime (length K)
        sv_rho: persistence of log-vol (vol clustering)
        sv_sigma: innovation std of log-vol
        logh_mu_list: regime-specific mean level of log-vol (length K)
        """
        self.Q = np.asarray(Q, float)
        self.c_list = np.asarray(c_list, float)
        self.Phi = np.asarray(Phi, float)
        self.Sigma_list = np.asarray(Sigma_list, float)
        self.regime_names = regime_names
        self.T_days = T_days
        self.seed = seed
        self.burn_in_months = burn_in_months
        self.start_date = start_date
        self.k0 = k0
        self.verbose = verbose
        
        self.K = len(regime_names)
        self.N = self.c_list.shape[1]
        self.asset_names = asset_names or [f"Asset{i+1}" for i in range(self.N)]
        
        self.df_list = df_list
        self.sv_rho = sv_rho
        self.sv_sigma = sv_sigma
        self.logh_mu_list = logh_mu_list
        
        self.daily_returns_df = None
        self.monthly_returns_df = None
        self.daily_regimes = None
        self.monthly_regimes = None
        self.dates = None
        self.sample_months = None
        
    def _expand_regime_param(self, values, fill_value):
        values = np.asarray(values, float) if values is not None else None
        if values is None:
            return np.full(self.K, fill_value, dtype=float)
        if values.size < self.K:
            return np.pad(values, (0, self.K - values.size), constant_values=fill_value)
        return values[:self.K]
        
    def simulate(self):
        set_numpy_determinism(self.seed)
        rng = np.random.default_rng(self.seed)
    
        df_list = self._expand_regime_param(self.df_list, 8.0)
        logh_mu_list = self._expand_regime_param(self.logh_mu_list, -1.0)
        sv_rho = float(self.sv_rho)
        sv_sigma = float(self.sv_sigma)
    
        r_log, k_daily, k_month, dates, sample_months = simulate_rs_var1_monthly_regimes_RS_SV_T(
            T_days=self.T_days,
            Q=self.Q,
            c_list=self.c_list,
            Phi=self.Phi,
            Sigma_list=self.Sigma_list,
            k0=self.k0,
            burn_in_months=self.burn_in_months,
            rng=rng,
            start_date=self.start_date,
            df_list=df_list,
            sv_rho=sv_rho,
            sv_sigma=sv_sigma,
            logh_mu_list=logh_mu_list,
        )
    
        # Convert log returns to simple returns
        r_simple = np.exp(r_log) - 1.0
    
        # Store daily returns
        self.daily_returns_df = pd.DataFrame(
            r_simple,
            index=dates,
            columns=self.asset_names
        )
    
        # Store daily regimes (named)
        self.daily_regimes = (
            pd.Series(k_daily, index=dates, name="regime_int")
            .map(lambda x: self.regime_names[int(x)])
        )
        self.daily_regimes.name = "regime"
    
        # Compute monthly returns FIRST (defines the "month-end" index as last trading day)
        self.monthly_returns_df = compute_monthly_factor_returns_from_daily(
            self.daily_returns_df,
            factor_cols=self.asset_names
        )
    
        # Build monthly regimes indexed EXACTLY like monthly_returns_df.index
        # k_month is aligned to sample_months (PeriodIndex, monthly)
        k_month_ser = pd.Series(k_month, index=sample_months, name="regime_int")
    
        idx = self.monthly_returns_df.index  # last trading day timestamps
        self.monthly_regimes = (
            k_month_ser
            .reindex(idx.to_period("M"))          # month period -> regime int
            .set_axis(idx)                        # index = last trading day timestamps
            .map(lambda x: self.regime_names[int(x)])  # int -> regime name
        )
        self.monthly_regimes.name = "regime"
    
        self.dates = dates
        self.sample_months = sample_months
    
        # Optional sanity check: should match perfectly
        if self.verbose:
            ok = self.monthly_regimes.index.equals(self.monthly_returns_df.index)
            print(f"  Simulated {len(self.daily_returns_df):,} days across {len(self.monthly_returns_df)} months")
            print(f"  Monthly regime index aligned to monthly returns: {ok}")
            print(f"  Daily regime distribution: {self.daily_regimes.value_counts().to_dict()}")
    
        return self
    

    
    def save_simulation_data(self, filepath):
        """Save all simulation data to pickle"""
        data = {
            'daily_returns': self.daily_returns_df,
            'monthly_returns': self.monthly_returns_df,
            'daily_regimes': self.daily_regimes,
            'monthly_regimes': self.monthly_regimes,
            'dates': self.dates,
            'sample_months': self.sample_months,
            'parameters': {
                'Q': self.Q,
                'c_list': self.c_list,
                'Phi': self.Phi,
                'Sigma_list': self.Sigma_list,
                'regime_names': self.regime_names,
                'seed': self.seed,
                'asset_names': self.asset_names,
                'df_list': self.df_list,
                'sv_rho': self.sv_rho,
                'sv_sigma': self.sv_sigma,
                'logh_mu_list': self.logh_mu_list
            },
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        if self.verbose:
            print(f" Saved simulation data to {filepath}")
        
    @staticmethod
    def load_simulation_data(filepath):
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
        print(f" Loaded simulation data from {filepath}")
        return data

In [ ]:
class StrategyBacktester:

    
    def __init__(self, env, verbose: bool = False):
        self.env = env
        self.results = {}
        self.verbose = verbose

    def _log(self, msg: str):
        if self.verbose:
            print(msg)
        
    def run_volatility_management(self, name="VolManagement", ridge=1e-8, gamma=5.0, c_tc=0.0021):
        self._log(f"\n[{name}] Running...")
        
        result = Markowitz_with_turnover_TC_diffobj(
            daily_factors=self.env.daily_returns_df,
            factor_cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            nonneg=True,
            c_tc=c_tc,
            use_drift_turnover=True,
            bounded_b=False  # Allow volatility timing via b parameters
        )
        
        # Compute daily returns from monthly weights
        # check this. do we scale with Mkt-rf?
        daily_port_returns = daily_portfolio_returns_from_monthly_weights(
            self.env.daily_returns_df,
            result['weights']
        )
        
        # Get weights by regime
        weights_by_regime = self._compute_weights_by_regime_markowitz(
            result['weights']
        )
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['portfolio_returns_net'],
            'weights_monthly': result['weights'],
            'turnover': result['turnover'],
            'costs': result['costs'],
            'a_params': result['a'],
            'b_params': result['b'],
            'weights_by_regime': weights_by_regime,
            'type': 'VolManagement'
        }
        
        self._log(f"   Vol-timing params (b): {result['b'].round(4).to_dict()}")
        return self
    
    def run_markowitz(self, name="Markowitz", gamma=5.0, c_tc=0.0021, 
                      use_drift_turnover=True, bounded_b=False, ridge=1e-8):
        self._log(f"\n[{name}] Running...")
        
        result = Markowitz_with_turnover_TC_diffobj(
            daily_factors=self.env.daily_returns_df,
            factor_cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            nonneg=True,
            c_tc=c_tc,
            use_drift_turnover=use_drift_turnover,
            bounded_b=bounded_b
        )
        
        daily_port_returns = daily_portfolio_returns_from_monthly_weights(
            self.env.daily_returns_df,
            result['weights']
        )
        
        weights_by_regime = self._compute_weights_by_regime_markowitz(
            result['weights']
        )
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['portfolio_returns_net'],
            'weights_monthly': result['weights'],
            'turnover': result['turnover'],
            'costs': result['costs'],
            'a_params': result['a'],
            'b_params': result['b'],
            'weights_by_regime': weights_by_regime,
            'type': 'Markowitz'
        }
        
        self._log(f"  Avg monthly turnover: {result['turnover'].mean():.4f}")
        return self
    
    def run_equal_weight(self, name="EqualWeight"):
        self._log(f"\n[{name}] Running...")
        
        weights = pd.Series(
            1.0 / self.env.N, 
            index=self.env.asset_names
        )
        
        daily_port_returns = (self.env.daily_returns_df @ weights)
        monthly_port_returns = (1 + daily_port_returns).groupby(
            daily_port_returns.index.to_period("M")
        ).prod() - 1.0
        monthly_port_returns.index = monthly_port_returns.index.to_timestamp("M")
        
        weights_by_regime = pd.DataFrame(
            {regime: weights for regime in self.env.regime_names}
        ).T
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': monthly_port_returns,
            'weights': weights,
            'weights_by_regime': weights_by_regime,
            'type': 'EqualWeight'
        }
        
        return self
    
    def run_mv_simple(self, name="MV_Simple", gamma=5.0, ridge=1e-10, 
                      sum_to_one_constraint=True, long_only=True):
        self._log(f"\n[{name}] Running...")
        
        result = mv_simple(
            daily_ret=self.env.daily_returns_df,
            cols=self.env.asset_names,
            gamma=gamma,
            ridge=ridge,
            sum_to_one_constraint=sum_to_one_constraint,
            long_only=long_only
        )
        
        weights_series = result['weights']
        daily_port_returns = (self.env.daily_returns_df @ weights_series)
        
        weights_by_regime = pd.DataFrame(
            {regime: weights_series for regime in self.env.regime_names}
        ).T
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['port_monthly'],
            'weights': weights_series,
            'weights_by_regime': weights_by_regime,
            'type': 'MV_Simple'
        }
        
        self._log(f"   Weights: {weights_series.round(4).to_dict()}")
        return self
    
    def run_mw_cvar_simple(self, name="MV_CVaR", beta=0.95, gamma=1.0,
                           sum_to_one_constraint=True, long_only=True, 
                           upper_bound=1.0, solver="ECOS"):
        """Run mean-variance CVaR optimization (unconditional, no vol timing, no turnover costs)"""
        self._log(f"\n[{name}] Running...")
        
        try:
            import cvxpy as cp
        except ImportError:
            self._log("  ✗ CVXPY not installed. Skipping MV-CVaR strategy.")
            return self
        
        result = mw_cvar_simple(
            daily_ret=self.env.daily_returns_df,
            cols=self.env.asset_names,
            beta=beta,
            gamma=gamma,
            sum_to_one_constraint=sum_to_one_constraint,
            long_only=long_only,
            upper_bound=upper_bound,
            solver=solver,
            verbose=False
        )
        
        weights_series = result['weights']
        daily_port_returns = (self.env.daily_returns_df @ weights_series)
        
        weights_by_regime = pd.DataFrame(
            {regime: weights_series for regime in self.env.regime_names}
        ).T
        
        self.results[name] = {
            'daily_returns': daily_port_returns,
            'monthly_returns': result['port_monthly'],
            'weights': weights_series,
            'weights_by_regime': weights_by_regime,
            'cvar_loss': result['cvar_loss'],
            'type': 'MV_CVaR'
        }
        
        self._log(f"   Weights: {weights_series.round(4).to_dict()}")
        self._log(f"   CVaR (loss): {result['cvar_loss']:.6f}")
        return self
    
    def _compute_weights_by_regime_markowitz(self, weights_monthly):
        common = weights_monthly.index.intersection(self.env.monthly_regimes.index)
        
        regime_weights = []
        for regime in self.env.regime_names:
            mask = self.env.monthly_regimes.loc[common] == regime
            if mask.sum() == 0:
                continue
            
            avg_weights = weights_monthly.loc[common][mask].mean()
            
            weights = avg_weights.to_dict()
            weights['Regime'] = regime
            regime_weights.append(weights)
        
        return pd.DataFrame(regime_weights).set_index('Regime')
    
    def save_results(self, filepath):
        data = {
            'results': self.results,
            'environment': {
                'regime_names': self.env.regime_names,
                'asset_names': self.env.asset_names,
                'seed': self.env.seed
            }
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)
        if self.verbose:
            print(f"Saved backtest results to {filepath}")

## Run All Scenarios and Generate Comprehensive Tables

This section runs all three regime-switching scenarios and generates performance tables for comparison.

In [17]:
df_list = np.array([20.0, 10.0, 5.0])
logh_mu_list = np.array([-1.6, -1.3, -0.7])
sv_rho, sv_sigma = 0.97, 0.20

scenarios_config = [
    ("Bull_Bear", Q_bull_bear),
    ("Neutral_Bear", Q_neutral_bear),
    ("Bull_Neutral", Q_bull_neutral),
]

# Storage for all results
all_scenario_results = {}
all_performance_tables = {}
# TODO add the df list, sv rho sv sigm and logh mu list to the environment
for scenario_name, Q_matrix in scenarios_config:
    env = MarkovSimulationEnvironment(
        Q=Q_matrix,
        c_list=c_list,
        Phi=Phi_fixed,
        Sigma_list=Sigma_list,
        regime_names=regime_names,
        T_days=252*50,  # 70 years total: 50 years training + 20 years testing
        seed=123,
        burn_in_months=50,
        start_date="2000-01-03",
        k0=0,
        asset_names=["Asset1", "Asset2"],
        verbose=False,
        df_list=df_list,
        sv_rho=sv_rho,
        sv_sigma=sv_sigma,
        logh_mu_list=logh_mu_list
    )

    env.simulate()

    # Run backtest with all strategies
    backtester = StrategyBacktester(env, verbose=False)
    backtester.run_equal_weight("EqualWeight")
    backtester.run_mv_simple("MV_Simple", gamma=5.0)
    backtester.run_mw_cvar_simple("MV_CVaR", beta=0.95, gamma=5.0)
    backtester.run_volatility_management("VolManagement", gamma=5.0, c_tc=0.0021)
    backtester.run_markowitz("VolManagement_Unconditional", gamma=5.0, c_tc=0.0021,
                             use_drift_turnover=True, bounded_b=True)

    # 3. Monthly-return performance table
    portfolios = {name: data['monthly_returns'] for name, data in backtester.results.items()}
    perf_table = make_table_for_portfolios(portfolios, periods_per_year=12)

    print("\n" + "-"*60)
    print(f"MONTHLY-RETURN PERFORMANCE TABLE: {scenario_name.replace('_', '-')}")
    print("-"*60)
    print(perf_table.round(3).to_string())

    # 4. Minimal summary (Sharpe + Tail-Adj Sharpe)
    summary = pd.DataFrame({
        "Sharpe (ann.)": perf_table.loc["Sharpe (ann.)"],
        "Tail-Adj Sharpe (CVaR95)": perf_table.loc["Tail-Adj Sharpe (CVaR95)"]
    })
    print("\nSummary (risk-adjusted vs tail-adjusted):")
    print(summary.round(3).to_string())

    # Store for comparison
    all_scenario_results[scenario_name] = {
        'env': env,
        'backtester': backtester,
        'perf_table': perf_table
    }
    all_performance_tables[scenario_name] = perf_table

TypeError: MarkovSimulationEnvironment.__init__() got an unexpected keyword argument 'df_list'

In [ ]:
# ====================================================================
# COMPARATIVE ANALYSIS ACROSS SCENARIOS
# ====================================================================

# Compare Sharpe ratios across scenarios
sharpe_comparison = pd.DataFrame()
for scenario_name, results in all_performance_tables.items():
    sharpe_comparison[scenario_name] = results.loc['Sharpe (ann.)']

print("\n" + "-"*60)
print("SHARPE RATIO COMPARISON")
print("-"*60)
print(sharpe_comparison.round(3).to_string())

# Compare annualized returns across scenarios
return_comparison = pd.DataFrame()
for scenario_name, results in all_performance_tables.items():
    return_comparison[scenario_name] = results.loc['Ann. Mean (%)']

print("\n" + "-"*60)
print("ANNUALIZED RETURN COMPARISON (%)")
print("-"*60)
print(return_comparison.round(2).to_string())

# Compare CVaR across scenarios
cvar_comparison = pd.DataFrame()
for scenario_name, results in all_performance_tables.items():
    cvar_comparison[scenario_name] = results.loc['CVaR 95% (%)']

print("\n" + "-"*60)
print("CVaR 95% COMPARISON (%) - Lower is more negative tail risk")
print("-"*60)
print(cvar_comparison.round(3).to_string())

# Compare Avg Drawdown across scenarios
dd_comparison = pd.DataFrame()
for scenario_name, results in all_performance_tables.items():
    dd_comparison[scenario_name] = results.loc['Avg DD (%)']

print("\n" + "-"*60)
print("AVERAGE DRAWDOWN COMPARISON (%)")
print("-"*60)
print(dd_comparison.round(2).to_string())

## Interpretation of Results Across Scenarios

Based on the VAR(1) regime-switching model, here's what we expect to see:

## Targeted Scenarios: Demonstrating Strategy Strengths

**Notation (used throughout):**
- $r_t$: daily return
- $\mu$: mean daily return
- $\sigma$: standard deviation of daily returns
- Sharpe $= \frac{\mu}{\sigma} \sqrt{252}$
- Tail-Adj Sharpe (CVaR95) $= \frac{\mu}{|\mathrm{CVaR}_{0.95}|} \sqrt{252}$

We create 3 scenarios designed to show where each strategy excels:
1. **Scenario A: MV_Simple wins** — stable regimes, low volatility variation
2. **Scenario B: MV_CVaR wins** — rare but severe crashes (tail risk)
3. **Scenario C: VolManagement wins** — strong volatility swings + regime flips

In [ ]:
# load the results and test on 4 different test seeds / training seeds...

In [ ]:

# ------------------------------------------------------------
# SCENARIO A: MV_SIMPLE WINS (stable / near-Gaussian / low tail value)
# Why MV wins:
# - risk is stable and tail asymmetry is minimal
# - CVaR constraint adds little benefit; with a moderately high gamma in CVaR objective
#   it tends to become unnecessarily conservative and slightly worse Sharpe
# ------------------------------------------------------------

Q_stable = np.array([
    [0.97, 0.02, 0.01],
    [0.02, 0.96, 0.02],
    [0.02, 0.03, 0.95],
])

# Means: Asset1 has higher mean across regimes, Asset2 slightly lower mean but safer.
c_stable = np.array([
    [0.0018, 0.0012],   # Bull
    [0.0015, 0.0011],   # Neutral
    [0.0012, 0.0010],   # Bear (still positive)
])

# Low vol across all regimes (little to time, little tail risk)
Sigma_stable = np.array([
    [[0.00050, 0.00010],
     [0.00010, 0.00030]],
    [[0.00055, 0.00010],
     [0.00010, 0.00032]],
    [[0.00060, 0.00010],
     [0.00010, 0.00035]],
])

# ------------------------------------------------------------
# SCENARIO B: MV_CVaR WINS (rare-but-severe crash; MV underprices tail)
# Why CVaR wins:
# - Crash regime is rare (so MV still likes risky asset on average)
# - But crash is severe for Asset1 -> worst months dominate tail metrics
# - Bear is brief enough that lagged-vol timing is less reliable
# ------------------------------------------------------------

Q_crash = np.array([
    [0.992, 0.007, 0.001],  # Bear very rare
    [0.060, 0.930, 0.010],
    [0.200, 0.050, 0.750],  # Bear persistent (expected duration ~ 4 months)
])

c_crash = np.array([
    [0.0030, 0.0010],     # Bull: Asset1 much better
    [0.0010, 0.0008],
    [-0.1500, 0.0003],    # Bear: extreme crash Asset1
])

Sigma_crash = np.array([
    [[0.0006,  0.00010],
     [0.00010, 0.00040]],
    [[0.0012,  0.00020],
     [0.00020, 0.00090]],
    [[0.0600,  0.0040],   # Bear: very high vol, weak diversification
     [0.0040,  0.0020]],
])

# ------------------------------------------------------------
# SCENARIO C: VOL MANAGEMENT WINS (big volatility swings + persistent high vol)
# Why VolManagement wins:
# - volatility changes a lot and is persistent
# - high-vol regimes have lower Sharpe; scaling/tilting by lagged vol helps
# ------------------------------------------------------------

Q_vol_regime = np.array([
    [0.85, 0.10, 0.05],
    [0.15, 0.70, 0.15],
    [0.10, 0.15, 0.75],
])

c_vol_regime = np.array([
    [0.0080, 0.0010],   # Bull: Asset1 dominates
    [0.0030, 0.0030],   # Neutral
    [-0.0040, 0.0060],  # Bear: Asset2 dominates
])

Sigma_vol_regime = np.array([
    [[0.0002, 0.00003],
     [0.00003, 0.0002]],
    [[0.0015, 0.0003],
     [0.0003, 0.0012]],
    [[0.0100, 0.0020],
     [0.0020, 0.0060]],
])


In [ ]:

seeds = [53, 274, 1234, 89]

targeted_scenarios = [
    ("A_MV_Simple_Wins", Q_stable,     c_stable,     Sigma_stable),
    ("B_MV_CVaR_Wins",   Q_crash,      c_crash,      Sigma_crash),
    ("C_VolMgmt_Wins",   Q_vol_regime, c_vol_regime, Sigma_vol_regime),
]

# tie tolerance for winner selection (prevents coin-flip when numbers are basically identical)
TIE_EPS = 1e-6

def choose_winner(metric_values: pd.Series, prefer: str | None = None, eps: float = 1e-6) -> str:
    """
    Pick the winner, but if multiple are within eps of the max, optionally prefer a named strategy
    (or fall back to alphabetical for determinism).
    """
    maxv = float(metric_values.max())
    close = metric_values[metric_values >= maxv - eps]
    if len(close) == 1:
        return close.index[0]
    if prefer is not None and prefer in close.index:
        return prefer
    return sorted(close.index)[0]

targeted_results = {}
targeted_tables = {}

for scenario_name, Q_matrix, c_scenario, Sigma_scenario in targeted_scenarios:

    # CVaR settings (keep beta fixed for statistical meaning)
    if scenario_name == "A_MV_Simple_Wins":
        mv_cvar_beta = 0.95
        mv_cvar_gamma = 1.0             # make CVaR "over-care" about tails in a stable world
        winner_metric = "Sharpe (ann.)"
        expected_winner = "MV_Simple"
        tie_preference = "MV_Simple"     # if MV and CVaR are identical, interpret as "MV wins / CVaR adds nothing"
    elif scenario_name == "B_MV_CVaR_Wins":
        mv_cvar_beta = 0.95
        mv_cvar_gamma = 1.0
        winner_metric = "Tail-Adj Sharpe (CVaR95)"
        expected_winner = "MV_CVaR"
        tie_preference = "MV_CVaR"
    else:
        mv_cvar_beta = 0.95
        mv_cvar_gamma = 1.0
        winner_metric = "Sharpe (ann.)"
        expected_winner = "VolManagement"
        tie_preference = "VolManagement"

    perf_tables = []
    
        
    for seed in seeds:
        env = MarkovSimulationEnvironment(
            Q=Q_matrix,
            c_list=c_scenario,
            Phi=Phi_fixed,
            Sigma_list=Sigma_scenario,
            regime_names=regime_names,
            T_days=252*70,
            seed=seed,
            burn_in_months=50,
            start_date="2000-01-03",
            k0=0,
            asset_names=["Asset1", "Asset2"],
            verbose=False
        )
        env.simulate()
    
        backtester = StrategyBacktester(env, verbose=False)
        backtester.run_equal_weight("EqualWeight")
        backtester.run_mv_simple("MV_Simple", gamma=5.0)
        backtester.run_mw_cvar_simple("MV_CVaR", beta=mv_cvar_beta, gamma=mv_cvar_gamma)
        backtester.run_volatility_management("VolManagement", gamma=5.0, c_tc=0.0021)
        backtester.run_markowitz(
            "VolManagement_Unconditional",
            gamma=5.0,
            c_tc=0.0021,
            use_drift_turnover=True,
            bounded_b=True
        )
    
        portfolios = {name: data["monthly_returns"] for name, data in backtester.results.items()}
        perf_table = make_table_for_portfolios(portfolios, periods_per_year=12)
        perf_tables.append(perf_table)
    
    # ---- Mean performance table across seeds ----
    common_index = perf_tables[0].index
    common_cols  = perf_tables[0].columns
    perf_tables  = [t.reindex(index=common_index, columns=common_cols) for t in perf_tables]
    
    mean_table = sum(perf_tables) / len(perf_tables)
    
    print("\n" + "-" * 60)
    print(f"MEAN MONTHLY PERFORMANCE TABLE (across {len(seeds)} seeds): {scenario_name.replace('_',' ')}")
    print("-" * 60)
    print(mean_table.round(3).to_string())
    
    # (Optional) choose winner ONCE from the averaged table
    metric_values_mean = mean_table.loc[winner_metric]
    winner_mean = choose_winner(metric_values_mean, prefer=tie_preference, eps=TIE_EPS)
    print(f"\nWinner metric: {winner_metric}")
    print(f"Winner on mean table: {winner_mean}")
    
    targeted_results[scenario_name] = {
        "mean_perf_table": mean_table,
        "winner_metric": winner_metric,
        "winner_on_mean": winner_mean,
        "seeds": seeds,
        "mv_cvar_beta": mv_cvar_beta,
        "mv_cvar_gamma": mv_cvar_gamma,
    }
    targeted_tables[scenario_name] = mean_table

    
    